<a href="https://colab.research.google.com/github/Jihyun13579/ESAA/blob/main/OBProject2_2129014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이콘 Basic 쇼핑몰 리뷰 평점 분류 경진대회
## https://dacon.io/competitions/official/235938/overview/description

#### 2129014 박지현

## 0. 한국어 분석을 위한 패키지 가져오기

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 44.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd

## 1. 데이터 불러오기


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA 학회_OB/OB_Project2/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA 학회_OB/OB_Project2/data/test.csv')

In [5]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA 학회_OB/OB_Project2/data/sample_submission.csv')

## 2. 데이터 살펴보기

### 1) train 데이터셋

In [6]:
print(train.shape)
train.head()

(25000, 3)


,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       25000 non-null  int64 
 1   reviews  25000 non-null  object
 2   target   25000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 586.1+ KB


In [8]:
train['target'].value_counts()

5    10000
2     8000
1     4500
4     2500
Name: target, dtype: int64

### test 데이터

In [9]:
print(test.shape)
test.head()

(25000, 2)


,id,reviews
0,0,채소가 약간 시들어 있어요
1,1,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...
2,2,부들부들 좋네요 입어보고 시원하면 또 살게요
3,3,이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?
4,4,검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       25000 non-null  int64 
 1   reviews  25000 non-null  object
dtypes: int64(1), object(1)
memory usage: 390.8+ KB


## 3. 데이터 전처리

In [11]:
import re

train = train.fillna(' ')
# 정규 표현식을 이용해 숫자를 공백으로 변경 (정규 표현식으로 \d는 숫자를 의미)
train['reviews'] = train['reviews'].apply(lambda x: re.sub(r'\d+', ' ', x))

# 테스트 데이터 세트: 동일하게 null 및 숫자를 공백으로 변환
test = test.fillna(' ')
test['reviews'] = test['reviews'].apply(lambda x: re.sub(r'\d+', ' ', x))

# id 칼럼 삭제 수행 (분석에 의미 없음)
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [12]:
from konlpy.tag import Twitter   # 감성 분석에 주로 사용됨

twitter = Twitter()
def tw_tokenizer(text):
  # 입력 인자로 들어온 텍스트를 형태소 단어로 토큰화해 리스트 형태로 반환
  tokens_ko = twitter.morphs(text)
  return tokens_ko

/usr/local/lib/python3.10/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


#### 피처 벡터화 위해 TF-IDF 사용

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# twitter 객체의 morphs() 객체를 이용한 tokenizer 사용. ngram_range=(1,2)
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train['reviews'])
tfidf_matrix_train = tfidf_vect.transform(train['reviews'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [15]:
tfidf_matrix_train

<25000x27796 sparse matrix of type '<class 'numpy.float64'>'
	with 488788 stored elements in Compressed Sparse Row format>